# Tutorial for Database Features in MLCBase

[![PyPI](https://img.shields.io/pypi/v/mlcbase)](https://pypi.org/project/mlcbase/) &nbsp;
[![license](https://img.shields.io/github/license/wmchen/mlcbase.svg)](https://www.apache.org/licenses/LICENSE-2.0)

Author: [Weiming Chen](https://weimingchen.net) and [Yuanshuang Sun](https://www.mulingcloud.com/author/yuanshuang-sun/)

## Introduction

This tutorial offers some examples to help you get familiar with our database APIs quickly.

Currently supported database backend:
- MySQL
- SQLite

Other backends such as PostgreSQL, MariaDB, MongoDB, Oracle, Redis, etc. are on the considered list.

In [1]:
import sys
sys.path.append("../src")
from datetime import datetime
from mlcbase import Logger, MySQLAPI, SQLiteAPI

logger = Logger()
logger.init_logger()


👋 Welcome to use MuLingCloud. We aim to let everything easier.

📍 mlcbase (1.2.4) imported



## 1. MySQL

MySQL is the most popular open source database backend. It can be deployed locally or remotely.

### 1.1 Connect to MySQL

You can establish an MySQL connection by instantiating `MySQLAPI()`.

In [2]:
host = ""      # change to your host address
port = 3306    # change to your port number, should be int
user = ""      # change to your login username
database = ""  # change to the name of your database
password = ""  # change to your login password
charset = ""   # change to the charset of your database, defaults to utf8

In [3]:
db_api = MySQLAPI(host, port, user, database, password, charset, logger=logger)

2024-10-30 19:28:17 | 0 day(s) 00:00:00 | INFO | connecting to database...
2024-10-30 19:28:18 | 0 day(s) 00:00:00 | SUCCESS | database connected


### 1.2 Ping the connection

You can ping the connection by calling `ping()`. It will not provide any feedback if pings successfully, otherwise, it will try to reconnect to the database.

It will be helpful if you have not operated the database for a long time.

In [4]:
db_api.ping()

### 1.3 Create data table

You can create a data table by calling `create_table()`.

Here is an example as follows.

Suppose you want to create a data table named "user" like:

|  id  | name | age  | gender | add_date |
| :--: | :--: | :--: | :----: | :------: |
| ...  | ...  | ...  |  ...   |   ...    |

#### Arguments

| args | type | remark |
| :-- | :-- | :---- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `table_config` | Optional[List[dict]] | Configurations of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `exist_ok` | bool | Create table if it exists. Defaults to True |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to use the combination of `table_name` and `table_config` to create a data table if you are not familiar with SQL language. 

The `table_config` is a list of dictionary which including the configuration of all fields in the table. An example of `table_config` is as follows.

```python
table_config = [
    dict(
        name="id",           # (required) the name of the field
        dtype="int",         # (required) the data type of the field
        not_null=True,       # (optional) whether the data of the field can be `NULL`. Defaults to False
        primary_key=True,    # (optional) whether the field is the primary key of the table. Defaults to False
        auto_increment=True  # (optional) whether the data in the field can increase automatically. Defaults to False
    ),
    dict(name="name", dtype="varchar(255)", not_null=True),
    dict(name="age", dtype="integer", not_null=True),
    dict(name="gender", dtype="integer", not_null=True, default="Unknown"),
    dict(name="add_date", dtype="date")
]
```

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `exist_ok=True` to avoid raising the error caused by creating an existing data table. (Only taking effect when you are using the combination of `table_name` and `table_config`)

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns True if success, otherwise return False

In [5]:
# create a data table named "user"
# the table has five fields, including id, name, age, gender and add_date
# id is the primary key of the table, and the data in this field can increase automatically
# id, name, age, and gender are not null fields, while add_date field can be NULL
# the default vaule of gender field is "Unknown"
db_api.create_table(table_name="user", 
                    table_config=[dict(name="id", dtype="int", not_null=True, primary_key=True, auto_increment=True),
                                  dict(name="name", dtype="varchar(255)", not_null=True),
                                  dict(name="age", dtype="integer", not_null=True),
                                  dict(name="gender", dtype="varchar(255)", default="Unknown", not_null=True),
                                  dict(name="add_date", dtype="date")])

2024-10-30 19:28:18 | 0 day(s) 00:00:00 | INFO | creating table...
2024-10-30 19:28:18 | 0 day(s) 00:00:00 | INFO | getting tables from database...
2024-10-30 19:28:18 | 0 day(s) 00:00:00 | SUCCESS | table created


True

### 1.4 Get tables

You can get the name of all tables in the database by calling `get_tables()`

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

You can print the result in the terminal as a table by setting `show=True` (defaults to False).

If you want to get tables with additional conditions, please provide the `sql_command` directly.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns table names in tuple if success, otherwise return None

In [6]:
db_api.get_tables(show=True)

2024-10-30 19:28:18 | 0 day(s) 00:00:00 | INFO | getting tables from database...


Show Data 
  Tables  
┏━━━━━━━━┓
┃ Tables ┃
┡━━━━━━━━┩
│  user  │
└────────┘

(('user',),)

### 1.5 Get fields

You can get the information of all fields in a specific data table by calling `get_fields()`.

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to specify `table_name` to get fields if you are not familiar with SQL language.

You can print the result in the terminal as a table by setting `show=True` (defaults to False).

If you want to get fields with additional conditions, please provide the `sql_command` directly.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns the information of fields in tuple if success, otherwise return None

In [7]:
db_api.get_fields(table_name="user", show=True)

2024-10-30 19:28:18 | 0 day(s) 00:00:00 | INFO | getting fields from data table...


                       Show Fields of 'user'                       
┏━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃  Field   ┃     Type     ┃ Null ┃ Key ┃ Default ┃     Extra      ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│    id    │   int(11)    │  NO  │ PRI │         │ auto_increment │
├──────────┼──────────────┼──────┼─────┼─────────┼────────────────┤
│   name   │ varchar(255) │  NO  │     │         │                │
├──────────┼──────────────┼──────┼─────┼─────────┼────────────────┤
│   age    │   int(11)    │  NO  │     │         │                │
├──────────┼──────────────┼──────┼─────┼─────────┼────────────────┤
│  gender  │ varchar(255) │  NO  │     │ Unknown │                │
├──────────┼──────────────┼──────┼─────┼─────────┼────────────────┤
│ add_date │     date     │ YES  │     │         │                │
└──────────┴──────────────┴──────┴─────┴─────────┴────────────────┘

(('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('name', 'varchar(255)', 'NO', '', None, ''),
 ('age', 'int(11)', 'NO', '', None, ''),
 ('gender', 'varchar(255)', 'NO', '', 'Unknown', ''),
 ('add_date', 'date', 'YES', '', None, ''))

### 1.6 Insert data

You can insert data into the table by calling `insert_data()`.

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `data` | Optional[dict] | Data waiting to be inserted. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to use the combination of `table_name` and `data` to insert data if you are not familiar with SQL language.

The `data` is a dictionary that specifies the correspondence of fields and values, in which the key of `data` is the field's name and the value of `data` is the inserting data of the corresponding field.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns True if success, otherwise return False

In [8]:
user_data = [dict(name="Weiming Chen", age=27, gender="male", add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="John", age=16, add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="David", age=45, add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="Peter", age=35)]

for data in user_data:
    db_api.insert_data(table_name="user", data=data)

2024-10-30 19:28:18 | 0 day(s) 00:00:01 | INFO | inserting data...
2024-10-30 19:28:18 | 0 day(s) 00:00:01 | SUCCESS | data inserted
2024-10-30 19:28:18 | 0 day(s) 00:00:01 | INFO | inserting data...
2024-10-30 19:28:18 | 0 day(s) 00:00:01 | SUCCESS | data inserted
2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | inserting data...
2024-10-30 19:28:19 | 0 day(s) 00:00:01 | SUCCESS | data inserted
2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | inserting data...
2024-10-30 19:28:19 | 0 day(s) 00:00:01 | SUCCESS | data inserted


### 1.7 Search data

You can search data from the table by calling `search_data()`.

Here are some examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `fields` | Optional[Union[List[str], str]] | The fields wish to return. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `list_all` | bool | Whether to return all data. Defaults to False |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend to use the combination of `table_name`, `fields`, `condition` and `list_all` to search data if you are not familiar with SQL language.

The `condition` arguments should be follow the syntax of SQL language if is not None. 

NOTE that if your condition parameter is a string, we highly recommend you to add "BINARY" keyword in your condition to avoid ambiguity.

You can print the result in the terminal as a table by setting `show=True`. (Would not take effect when you provide `sql_command`)

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns data in tuple if success, otherwise return None.

In [9]:
# Case 1: only return the field of "name", and the condition is searching the users with age range from 18 to 30
data = db_api.search_data(table_name="user", fields="name", condition="age BETWEEN 18 AND 30", show=True)
print(data)

2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | searching data...


  Show Data in  
     'user'     
┏━━━━━━━━━━━━━━┓
┃     name     ┃
┡━━━━━━━━━━━━━━┩
│ Weiming Chen │
└──────────────┘

(('Weiming Chen',),)


In [10]:
# Case 2: return the fields of "name" and "age", and the condition is searching the users whose age is less than or equal to 18.
data = db_api.search_data(table_name="user", fields=["name", "age"], condition="age<=18", show=True)
print(data)

2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | searching data...


 Show Data in 
    'user'    
┏━━━━━━┳━━━━━┓
┃ name ┃ age ┃
┡━━━━━━╇━━━━━┩
│ John │ 16  │
└──────┴─────┘

(('John', 16),)


In [11]:
# Case 3: return the fields of "name", "age" and "add_date", and return all data
data = db_api.search_data(table_name="user", fields=["name", "age", "add_date"], list_all=True, show=True)
print(data)

2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | searching data...


        Show Data in 'user'        
┏━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━┓
┃     name     ┃ age ┃  add_date  ┃
┡━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━┩
│ Weiming Chen │ 27  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│     John     │ 16  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│    David     │ 45  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│    Peter     │ 35  │    None    │
└──────────────┴─────┴────────────┘

(('Weiming Chen', 27, datetime.date(2024, 10, 30)), ('John', 16, datetime.date(2024, 10, 30)), ('David', 45, datetime.date(2024, 10, 30)), ('Peter', 35, None))


In [12]:
# Case 4: return all fields and all data
data = db_api.search_data(table_name="user", list_all=True, show=True)
print(data)

2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | searching data...
2024-10-30 19:28:19 | 0 day(s) 00:00:01 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 27  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

((1, 'Weiming Chen', 27, 'male', datetime.date(2024, 10, 30)), (2, 'John', 16, 'Unknown', datetime.date(2024, 10, 30)), (3, 'David', 45, 'Unknown', datetime.date(2024, 10, 30)), (4, 'Peter', 35, 'Unknown', None))


### 1.8 Update data

You can update data in the table by calling `update_data()`.

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `data` | Optional[dict] | Data waiting to be updated. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to use the combination of `table_name`, `data` and `condition` to update data if you are not familiar with SQL language.

The `data` is a dictionary that specifies the correspondence of fields and values, in which the key of `data` is the field's name and the value of `data` is the updating data of the corresponding field.

The `condition` arguments should be follow the syntax of SQL language if is not None.

NOTE that if your condition parameter is a string, we highly recommend you to add "BINARY" keyword in your condition to avoid ambiguity for the reason that MySQL is not sensitive with the case of letters.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns True if success, otherwise return False

In [13]:
db_api.update_data(table_name="user", data=dict(age=18), condition="BINARY name='Weiming Chen'")

# show data
db_api.search_data(table_name="user", list_all=True, show=True)

2024-10-30 19:28:19 | 0 day(s) 00:00:02 | INFO | updating data...
2024-10-30 19:28:19 | 0 day(s) 00:00:02 | SUCCESS | data updated
2024-10-30 19:28:19 | 0 day(s) 00:00:02 | INFO | searching data...
2024-10-30 19:28:19 | 0 day(s) 00:00:02 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 18  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

((1, 'Weiming Chen', 18, 'male', datetime.date(2024, 10, 30)),
 (2, 'John', 16, 'Unknown', datetime.date(2024, 10, 30)),
 (3, 'David', 45, 'Unknown', datetime.date(2024, 10, 30)),
 (4, 'Peter', 35, 'Unknown', None))

In [14]:
db_api.insert_data(table_name="user", data=dict(name="weiming chen", age=27, gender="male"))
db_api.search_data(table_name="user", list_all=True, show=True)

# if you don't add the "BINARY" keyword may cause ambiguity
print("\nNow updating data without 'BINARY' keyword...\n")
db_api.update_data(table_name="user", data=dict(age=20), condition="name='Weiming Chen'")
db_api.search_data(table_name="user", list_all=True, show=True)
print("Note that both ages of 'Weiming Chen' and 'weiming chen' changed to 20, but I only want to change the age of 'Weiming Chen'")

2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | inserting data...
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | SUCCESS | data inserted
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | searching data...
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 18  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 27  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | updating data...
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | SUCCESS | data updated
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | searching data...
2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | getting fields from data table...



Now updating data without 'BINARY' keyword...



               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 20  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 20  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

Note that both ages of 'Weiming Chen' and 'weiming chen' changed to 20, but I only want to change the age of 'Weiming Chen'


### 1.9 Delete data

You can delete data from the table by calling `delete_data()`.

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to use the combination of `table_name` and `condition` to delete data if you are not familiar with SQL language.

The `condition` arguments should be follow the syntax of SQL language if is not None.

NOTE that if your condition parameter is a string, we highly recommend you to add "BINARY" keyword in your condition to avoid ambiguity.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns True if success, otherwise return False

In [15]:
db_api.delete_data(table_name="user", condition="age>30")

# show data
db_api.search_data(table_name="user", list_all=True, show=True)

2024-10-30 19:28:20 | 0 day(s) 00:00:02 | INFO | deleting data...
2024-10-30 19:28:20 | 0 day(s) 00:00:03 | SUCCESS | data deleted
2024-10-30 19:28:20 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:20 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 20  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 20  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

((1, 'Weiming Chen', 20, 'male', datetime.date(2024, 10, 30)),
 (2, 'John', 16, 'Unknown', datetime.date(2024, 10, 30)),
 (5, 'weiming chen', 20, 'male', None))

### 1.10 Delete data table

You can delete the entire data table by calling `delete_table()`

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `not_exist_ok` | bool | Delete table if it not exists. Default to True |
| `use_ping` | bool | Ping the connection before operation. Defaults to True |

We recommend you to specify `table_name` to get fields if you are not familiar with SQL language.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `not_exist_ok=True` to avoid raising the error caused by deleting an non-existing data table. (Only taking effect when `table_name` is specified)

We defaults to set `use_ping=True` to ping the database connection before operation.

#### Returns

It returns True if success, otherwise return False

In [16]:
db_api.delete_table(table_name="user")

# show table
db_api.get_tables(show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | deleting data table...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | table deleted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...


Show Data 
  Tables  
┏━━━━━━━━┓
┃ Tables ┃
┡━━━━━━━━┩
└────────┘

()

### 1.11 Close connection

After using the database, don't forget to close the connection by calling `close()`.

In [17]:
db_api.close()

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | database connection closed


## 2. SQLite

SQLite is a widely used light-weight local database backend. The usage of `SQLiteAPI` is highly similar to `MySQLAPI` which only has a little difference.

### 2.1 Open SQLite database

You can open an SQLite database by instantiating `SQLiteAPI()`.

If you want to open a database on dist, you can set `in_memory=False` and provide `db_path`.

If you want to create a database only existing in memory, you can set `in_memory=True`.

In [18]:
db_api = SQLiteAPI(in_memory=True, logger=logger)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | connecting to database...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | database connected


### 2.2 Create data table

You can create a data table by calling `create_table()`.

Here is an example as follows.

Suppose you want to create a data table named "user" like:

|  id  | name | age  | gender | add_date |
| :--: | :--: | :--: | :----: | :------: |
| ...  | ...  | ...  |  ...   |   ...    |

NOTE that the diiference between SQLite and MySQL that SQLite cannot create a table with multiple primary keys and auto-increment primary key at the same time, while MySQL can.

#### Arguments

| args | type | remark |
| :-- | :-- | :---- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `table_config` | Optional[List[dict]] | Configurations of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `exist_ok` | bool | Create table if it exists. Defaults to True |

We recommend you to use the combination of `table_name` and `table_config` to create a data table if you are not familiar with SQL language. 

The `table_config` is a list of dictionary which including the configuration of all fields in the table. An example of `table_config` is as follows.

```python
table_config = [
    dict(
        name="id",           # (required) the name of the field
        dtype="int",         # (required) the data type of the field
        not_null=True,       # (optional) whether the data of the field can be `NULL`. Defaults to False
        primary_key=True,    # (optional) whether the field is the primary key of the table. Defaults to False
        auto_increment=True  # (optional) whether the data in the field can increase automatically. Defaults to False
    ),
    dict(name="name", dtype="varchar(255)", not_null=True),
    dict(name="age", dtype="integer", not_null=True),
    dict(name="gender", dtype="integer", not_null=True, default="Unknown"),
    dict(name="add_date", dtype="date")
]
```

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `exist_ok=True` to avoid raising the error caused by creating an existing data table. (Only taking effect when you are using the combination of `table_name` and `table_config`)

#### Returns

It returns True if success, otherwise return False

In [19]:
# create a data table named "user"
# the table has five fields, including id, name, age, gender and add_date
# id is the primary key of the table, and the data in this field can increase automatically
# id, name, age, and gender are not null fields, while add_date field can be NULL
# the default vaule of gender field is "Unknown"
db_api.create_table(table_name="user",
                    table_config=[dict(name="id", dtype="integer", not_null=True, primary_key=True, auto_increment=True),
                                  dict(name="name", dtype="varchar(255)", not_null=True),
                                  dict(name="age", dtype="integer", not_null=True),
                                  dict(name="gender", dtype="varchar(255)", default="Unknown", not_null=True),
                                  dict(name="add_date", dtype="date")])

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | creating table...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | table created


True

### 2.3 Get tables

You can get the name of all tables in the database by calling `get_tables()`

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |
| `return_sqlite_sequence` | bool | whether to return the "sqlite_sequence" table. Defaults to False |

You can print the result in the terminal as a table by setting `show=True` (defaults to False).

If you want to get tables with additional conditions, please provide the `sql_command` directly.

We defaults to set `return_sqlite_sequence=False` to filter the "sqlite_sequence" table.

#### Returns

It returns table names in tuple if success, otherwise return None

In [20]:
db_api.get_tables(show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...


Show Data 
  Tables  
┏━━━━━━━━┓
┃ Tables ┃
┡━━━━━━━━┩
│  user  │
└────────┘

(('user',),)

In [21]:
db_api.get_tables(show=True, return_sqlite_sequence=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...


 Show Data Tables  
┏━━━━━━━━━━━━━━━━━┓
┃     Tables      ┃
┡━━━━━━━━━━━━━━━━━┩
│      user       │
├─────────────────┤
│ sqlite_sequence │
└─────────────────┘

(('user',), ('sqlite_sequence',))

### 2.4 Get fields

You can get the information of all fields in a specific data table by calling `get_fields()`.

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |

We recommend you to specify `table_name` to get fields if you are not familiar with SQL language.

You can print the result in the terminal as a table by setting `show=True` (defaults to False).

If you want to get fields with additional conditions, please provide the `sql_command` directly.

#### Returns

It returns the information of fields in tuple if success, otherwise return None

In [22]:
db_api.get_fields(table_name="user", show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


                          Show Fields of 'user'                          
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Column ┃  Field   ┃     Type     ┃ Not Null ┃  Default  ┃ Primary Key ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│   0    │    id    │   INTEGER    │    1     │   None    │      1      │
├────────┼──────────┼──────────────┼──────────┼───────────┼─────────────┤
│   1    │   name   │ VARCHAR(255) │    1     │   None    │      0      │
├────────┼──────────┼──────────────┼──────────┼───────────┼─────────────┤
│   2    │   age    │   INTEGER    │    1     │   None    │      0      │
├────────┼──────────┼──────────────┼──────────┼───────────┼─────────────┤
│   3    │  gender  │ VARCHAR(255) │    1     │ 'Unknown' │      0      │
├────────┼──────────┼──────────────┼──────────┼───────────┼─────────────┤
│   4    │ add_date │     DATE     │    0     │   None    │      0      │
└────────┴──────────┴──────────────┴──────────┴───────────┴─────────────┘

((0, 'id', 'INTEGER', 1, None, 1),
 (1, 'name', 'VARCHAR(255)', 1, None, 0),
 (2, 'age', 'INTEGER', 1, None, 0),
 (3, 'gender', 'VARCHAR(255)', 1, "'Unknown'", 0),
 (4, 'add_date', 'DATE', 0, None, 0))

### 2.5 Insert data

You can insert data into the table by calling `insert_data()`.

Here is an example as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `data` | Optional[dict] | Data waiting to be inserted. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |

We recommend you to use the combination of `table_name` and `data` to insert data if you are not familiar with SQL language.

The `data` is a dictionary that specifies the correspondence of fields and values, in which the key of `data` is the field's name and the value of `data` is the inserting data of the corresponding field.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

#### Returns

It returns True if success, otherwise return False

In [23]:
user_data = [dict(name="Weiming Chen", age=27, gender="male", add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="John", age=16, add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="David", age=45, add_date=datetime.now().strftime("%Y-%m-%d")),
             dict(name="Peter", age=35)]

for data in user_data:
    db_api.insert_data(table_name="user", data=data)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | inserting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data inserted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | inserting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data inserted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | inserting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data inserted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | inserting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data inserted


### 2.6 Search data

You can search data from the table by calling `search_data()`.

Here are some examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `fields` | Optional[Union[List[str], str]] | The fields wish to return. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `list_all` | bool | Whether to return all data. Defaults to False |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `show` | bool | Print the returns in the terminal. Defaults to False |

We recommend to use the combination of `table_name`, `fields`, `condition` and `list_all` to search data if you are not familiar with SQL language.

The `condition` arguments should be follow the syntax of SQL language if is not None. 

NOTE that if your condition parameter is a string, we highly recommend you to add "BINARY" keyword in your condition to avoid ambiguity.

You can print the result in the terminal as a table by setting `show=True`. (Would not take effect when you provide `sql_command`)

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

#### Returns

It returns data in tuple if success, otherwise return None.

In [24]:
# Case 1: only return the field of "name", and the condition is searching the users with age range from 18 to 30
data = db_api.search_data(table_name="user", fields="name", condition="age BETWEEN 18 AND 30", show=True)
print(data)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...


  Show Data in  
     'user'     
┏━━━━━━━━━━━━━━┓
┃     name     ┃
┡━━━━━━━━━━━━━━┩
│ Weiming Chen │
└──────────────┘

(('Weiming Chen',),)


In [25]:
# Case 2: return the fields of "name" and "age", and the condition is searching the users whose age is less than or equal to 18.
data = db_api.search_data(table_name="user", fields=["name", "age"], condition="age<=18", show=True)
print(data)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...


 Show Data in 
    'user'    
┏━━━━━━┳━━━━━┓
┃ name ┃ age ┃
┡━━━━━━╇━━━━━┩
│ John │ 16  │
└──────┴─────┘

(('John', 16),)


In [26]:
# Case 3: return the fields of "name", "age" and "add_date", and return all data
data = db_api.search_data(table_name="user", fields=["name", "age", "add_date"], list_all=True, show=True)
print(data)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...


        Show Data in 'user'        
┏━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━┓
┃     name     ┃ age ┃  add_date  ┃
┡━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━┩
│ Weiming Chen │ 27  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│     John     │ 16  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│    David     │ 45  │ 2024-10-30 │
├──────────────┼─────┼────────────┤
│    Peter     │ 35  │    None    │
└──────────────┴─────┴────────────┘

(('Weiming Chen', 27, '2024-10-30'), ('John', 16, '2024-10-30'), ('David', 45, '2024-10-30'), ('Peter', 35, None))


In [27]:
# Case 4: return all fields and all data
data = db_api.search_data(table_name="user", list_all=True, show=True)
print(data)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 27  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

((1, 'Weiming Chen', 27, 'male', '2024-10-30'), (2, 'John', 16, 'Unknown', '2024-10-30'), (3, 'David', 45, 'Unknown', '2024-10-30'), (4, 'Peter', 35, 'Unknown', None))


### 2.7 Update data

You can update data in the table by calling `update_data()`.

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `data` | Optional[dict] | Data waiting to be updated. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |

We recommend you to use the combination of `table_name`, `data` and `condition` to update data if you are not familiar with SQL language.

The `data` is a dictionary that specifies the correspondence of fields and values, in which the key of `data` is the field's name and the value of `data` is the updating data of the corresponding field.

The `condition` arguments should be follow the syntax of SQL language if is not None.

NOTE that SQLite is sensitive with the case of letters, you do not need to specify the "BINARY" keyword as in MySQL. (Besides, SQLite does not support the "BINARY" keyword)

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

#### Returns

It returns True if success, otherwise return False

In [28]:
db_api.update_data(table_name="user", data=dict(age=18), condition="name='Weiming Chen'")

# show data
db_api.search_data(table_name="user", condition="name='Weiming Chen'", show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | updating data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data updated
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'               
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 18  │  male  │ 2024-10-30 │
└────┴──────────────┴─────┴────────┴────────────┘

((1, 'Weiming Chen', 18, 'male', '2024-10-30'),)

In [29]:
db_api.insert_data(table_name="user", data=dict(name="weiming chen", age=27, gender="male"))
db_api.search_data(table_name="user", list_all=True, show=True)

# SQLite is sensitive with the case of letters
db_api.update_data(table_name="user", data=dict(age=20), condition="name='Weiming Chen'")
db_api.search_data(table_name="user", list_all=True, show=True)
print("SQLite is sensitive with the case of letters, only the age of 'Weiming Chen' is changed to 20")

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | inserting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data inserted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 18  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 27  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | updating data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data updated
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 20  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 3  │    David     │ 45  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 4  │    Peter     │ 35  │ Unknown │    None    │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 27  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

SQLite is sensitive with the case of letters, only the age of 'Weiming Chen' is changed to 20


### 2.8 Delete data

You can delete data from the table by calling `delete_data()`.

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `condition` | Optional[str] | The searching condition. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |

We recommend you to use the combination of `table_name` and `condition` to delete data if you are not familiar with SQL language.

The `condition` arguments should be follow the syntax of SQL language if is not None.

NOTE that if your condition parameter is a string, we highly recommend you to add "BINARY" keyword in your condition to avoid ambiguity.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

#### Returns

It returns True if success, otherwise return False

In [30]:
db_api.delete_data(table_name="user", condition="age>30")

# show data
db_api.search_data(table_name="user", list_all=True, show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | deleting data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | data deleted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | searching data...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting fields from data table...


               Show Data in 'user'                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id ┃     name     ┃ age ┃ gender  ┃  add_date  ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩
│ 1  │ Weiming Chen │ 20  │  male   │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 2  │     John     │ 16  │ Unknown │ 2024-10-30 │
├────┼──────────────┼─────┼─────────┼────────────┤
│ 5  │ weiming chen │ 27  │  male   │    None    │
└────┴──────────────┴─────┴─────────┴────────────┘

((1, 'Weiming Chen', 20, 'male', '2024-10-30'),
 (2, 'John', 16, 'Unknown', '2024-10-30'),
 (5, 'weiming chen', 27, 'male', None))

### 2.9 Delete data table

You can delete the entire data table by calling `delete_table()`

Here is an examples as follows.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `table_name` | Optional[str] | The name of the table. Defaults to None |
| `sql_command` | Optional[str] | SQL command. Defaults to None |
| `not_exist_ok` | bool | Delete table if it not exists. Default to True |

We recommend you to specify `table_name` to get fields if you are not familiar with SQL language.

However, if you are familiar with SQL language, you can also directly provice `sql_command` for more flexible usage.

We defaults to set `not_exist_ok=True` to avoid raising the error caused by deleting an non-existing data table. (Only taking effect when `table_name` is specified)

#### Returns

It returns True if success, otherwise return False

In [31]:
db_api.delete_table(table_name="user")

# show table
db_api.get_tables(show=True)

2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | deleting data table...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | SUCCESS | table deleted
2024-10-30 19:28:21 | 0 day(s) 00:00:03 | INFO | getting tables from database...


Show Data 
  Tables  
┏━━━━━━━━┓
┃ Tables ┃
┡━━━━━━━━┩
└────────┘

()

### 2.10 Close connection

After using the database, don't forget to close the connection by calling `close()`.

In [32]:
db_api.close()

2024-10-30 19:28:21 | 0 day(s) 00:00:04 | INFO | database connection closed
